# Restart and resolution change

Fluidsim provides 2 command-line tools to restart a simulation (`fluidsim-restart`) 
and create a new file with a modified resolution (`fluidsim-modif-resolution`). 
Let's present these very useful tools!

First, one can get help by invoking these tools with the `-h` option:

In [ ]:
!fluidsim-restart -h

In [ ]:
!fluidsim-modif-resolution -h

In [ ]:
from pathlib import Path
import shutil

path_fls = "/tmp/sim_data"
path_base = Path(path_fls) / "doc_aniso"
shutil.rmtree(path_base, ignore_errors=True)

In [ ]:
import os

os.environ["FLUIDSIM_PATH"] = path_fls
os.environ["TRANSONIC_BACKEND"] = "python"

Then, let's use these tools. First we need a simulation directory. 

In [ ]:
from fluidsim.util.scripts.turb_trandom_anisotropic import main

params, sim = main(
    args=(
        '--sub-directory "doc_aniso" -nz 12 --ratio-nh-nz 2 --t_end 1 '
        '--modify-params "params.output.periods_print.print_stdout = 0.25; '
        'params.output.periods_save.spect_energy_budg = 0"'
    )
)

We then define a variable with the path of the directory containing the results of the simulation.

In [ ]:
sim.output.path_run

In [ ]:
from pprint import pprint


def ls_path_run(glob="*"):
    pprint(sorted(p.name for p in Path(sim.output.path_run).glob(glob)))


ls_path_run()

In [ ]:
sim.output.spatial_means.plot()

Ah, we see that `t_end = 1` was not enough to obtain a statistically steady state. We need to run the simulation longer. Let's restart it to run until `t_end = 2`:

In [ ]:
!fluidsim-restart {sim.output.path_run} --t_end 2.0

In [ ]:
ls_path_run()

In [ ]:
sim.output.spatial_means.plot()

Finally, we are not far from a statistically steady state. We can relaunch the simulation to save results about this state. Here we show how to restart a simulation with the spatiotemporal spectra output activated:

In [ ]:
!fluidsim-restart {sim.output.path_run} --t_end 3.0 --modify-params "params.output.periods_save.spatiotemporal_spectra = 2 * pi / (4 * params.N)"

In [ ]:
ls_path_run()

`spatiotemporal` is a directory containing the data that can be used to compute spatiotemporal spectra:

In [ ]:
ls_path_run("spatiotemporal/*")

In [ ]:
sim.output.spatial_means.plot()

We want to start a larger simulation from the last state of this small simulation. We first need to create a larger file:

In [ ]:
!fluidsim-modif-resolution {sim.output.path_run} 5/4

We can now launch the last simulation from the file just created.

In [ ]:
path_new_state = next(Path(sim.output.path_run).glob("State_phys*"))
str(path_new_state)

In [ ]:
!fluidsim-restart {path_new_state} --t_end 3.1 --modify-params "params.NEW_DIR_RESULTS = True; params.nu_2 /= 2; params.output.periods_print.print_stdout = 0.05; params.output.periods_save.spatial_means = 0.01;"

In [ ]:
[p.name for p in path_base.glob("*")]